In [ ]:
#import libraries

import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
# Setting up batch size
b_size = 32

# Setting up image size
i_size = 256

# Setting up channels 
channels=3

# Setting up epochs
epochs=50

In [ ]:
#loop through sub-categories
for directory_path in glob.glob("image_dataset"):
    #extract label from sub-directory
    label = directory_path.split("/")[-1]
    print(label)

    #loop through image files
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
      try: 
            
            #read images with openCV
            img = cv2.imread(img_path)

            #checking image type
            tip = imghdr.what(img_path)

            #remove the img file if not in list of img extension
            if tip not in image_exts: 
                print('Image not in list {}'.format(img_path))
                os.remove(img_path)
      except Exception as e: 
            
            #print error msg if img has error
            print('Issue with image {}'.format(img_path))


In [ ]:
#loop image cls in data_dir
for image_class in os.listdir(data_dir): 

    #loop img in img_cls_dir
    for image in os.listdir(os.path.join(data_dir, image_class)):

        #create img path
        img_paths = os.path.join(data_dir, image_class, image)
        try: 

            #read images with openCV
            img = cv2.imread(img_paths)

            #checking image type
            tip = imghdr.what(img_paths)

            #remove the img file if not in list of img extension
            if tip not in image_exts: 
                print('Image not in list {}'.format(img_paths))
                os.remove(img_paths)

        #print error msg if img has error
        except Exception as e: 
            print('Issue with image {}'.format(img_paths))


In [ ]:

#preprocessing images in directory
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "image_dataset",
    #shuffling the dataset
    shuffle=True,
    #resizing image size
    image_size=(i_size,i_size),
    #setting batch size
    batch_size=b_size
)

In [ ]:
#getting classes names
class_names = dataset.class_names
class_names

In [ ]:
len(dataset)

In [ ]:
#printing img_btch shape and labls
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

In [ ]:
#plotting images from dataset
plt.figure(figsize=(12, 12))
for image_batch, labels_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[labels_batch[i]])
        plt.axis("off")

In [ ]:
len(dataset)

In [ ]:
#80% --> train
#20% --> 10% validation, 10% test
#train_size ==> 0.8
batches_c=int(len(dataset)*0.8)

#validation_size==>0.1
val_c=int(len(dataset)*0.1)

#test_size==>0.1
test_c=int(len(dataset)*0.1)

In [ ]:
# Define function split dataset training set, validation set, test set

def get_dataset_partitions_tf(dataset_name, train_data_size, val_data_size, test_data_size, shuffle=True, shuffle_size=10000):

    # Calculate length of the dataset
    dataset_size = len(dataset_name)
    #check sum of the dataset is correct
    assert (train_data_size+val_data_size+test_data_size) == dataset_size
        
    #shuffle the dataset to randomize the order of the dataset
    if shuffle:
        dataset_name = dataset_name.shuffle(shuffle_size, seed=5)
    
    #split dataset training set, validation set, test set
    train_dataset = dataset_name.take(train_data_size)    
    validation_dataset = dataset_name.skip(train_data_size).take(val_data_size)
    test_dataset = dataset_name.skip(train_data_size).skip(test_data_size)
    
    return train_dataset, validation_dataset, test_dataset

In [ ]:
train_dataset, validation_dataset, test_dataset = get_dataset_partitions_tf(dataset,batches_c,val_c,test_c)

In [ ]:
len(train_dataset)

In [ ]:
len(validation_dataset)

In [ ]:
len(test_dataset)

In [ ]:
#optimize training performance
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
#resize and rescaling images
resize_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(i_size, i_size),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [ ]:
#define data augmentation 
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
#applying data augmentation to training dataset and prefetching data
train_dataset = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# defining the shape, classes count, and create model

input_shape = (b_size, i_size, i_size, channels)
n_classes = 3

model = models.Sequential([
    resize_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

# Build model with the specified input shape
model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_dataset,
    batch_size=b_size,
    validation_data=validation_dataset,
    verbose=1,
    epochs=50,
)

In [ ]:
scores = model.evaluate(test_dataset)

In [ ]:
scores

In [ ]:
history

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
type(history.history['loss'])

In [ ]:
len(history.history['loss'])

In [ ]:
history.history['loss'][:10]

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(epochs), acc, label='Training Accuracy')
plt.plot(range(epochs), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("first img predict")
    plt.imshow(first_image)
    print("actual :",class_names[first_label])
    
    batch_prediction = model.predict(images_batch)
    print("predicted :",class_names[np.argmax(batch_prediction[0])])

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

In [ ]:
import os
model_version=max([int(i) for i in os.listdir("../models") + [0]])+1
model.save(f"../models/{model_version}")

In [ ]:
model.save("../gratato_model.h5")